<h1><center><font color = 'blue'>Create 2D df CH7</font></center></h1>

In [1]:
import pandas as pd
import psutil
import time
import os

from math import trunc

Pandas consume a lot of RAM when dealing with large dataset, so let's define a function to keep an eye on how much we are using.

In [2]:
def print_ram_usage():
    process = psutil.Process(os.getpid())
    ram_usage = round(process.memory_info().rss/float(2**30), 2)
    print("RAM usage: {}GB".format(ram_usage))

### Import data

In [3]:
start = time.time()

df = [pd.read_hdf("../efs/data/amsua_n18_ch7_4pts_part1.h5"),
      pd.read_hdf("../efs/data/amsua_n18_ch7_4pts_part2.h5"),
      pd.read_hdf("../efs/data/amsua_n18_ch7_4pts_part3.h5")]

df = pd.concat(df, axis=0)

df.reset_index(drop=True, inplace=True)

end = time.time()
print("Data imported in: {} min and {} sec".format(trunc((end - start)/60),
                                                   round((end - start)%60)))

print_ram_usage()

Data imported in: 4 min and 41 sec
RAM usage: 37.25GB


### Compute sensitivity, and hour feature

In [4]:
df["SENSITIVITY"] = df["IMPACT"]/df["OMF"]
df["hour"] = df["DATETIME"].dt.hour

In [5]:
df.tail()

,DATETIME,IMPACT,LATITUDE,LONGITUDE,OMF,point1_delp_0.015,point1_delp_0.026,point1_delp_0.04,point1_delp_0.057,point1_delp_0.078,...,point4_v_873.975,point4_v_888.632,point4_v_903.289,point4_v_917.946,point4_v_932.602,point4_v_947.258,point4_v_961.913,point4_v_976.62,SENSITIVITY,hour
4199834,2015-02-28 18:00:00,-0.000004,88.250000,-96.519997,0.329822,1.0,1.27,1.488501,1.8415,2.3345,...,0.943486,1.406512,1.703229,1.840314,1.823031,2.386230,3.894594,4.876057,-0.000011,18
4199835,2015-02-28 18:00:00,-0.000165,88.419998,111.849998,0.196722,1.0,1.27,1.488501,1.8415,2.3345,...,-4.071671,-4.052464,-4.003406,-3.326711,-2.582695,-2.646241,-2.641682,-2.435990,-0.000839,18
4199836,2015-02-28 18:00:00,-0.000078,89.189995,-171.569992,0.156673,1.0,1.27,1.488501,1.8415,2.3345,...,-6.099522,-5.733474,-5.448677,-5.184715,-5.634592,-6.047100,-6.297201,-6.204467,-0.000495,18
4199837,2015-02-28 18:00:00,-0.000063,89.269997,46.529999,0.084857,1.0,1.27,1.488501,1.8415,2.3345,...,7.171109,7.229550,7.014696,6.663498,6.144629,5.549446,4.969238,4.628523,-0.000743,18
4199838,2015-02-28 18:00:00,-0.000064,89.320000,-62.049999,0.129242,1.0,1.27,1.488501,1.8415,2.3345,...,1.447804,1.013766,0.806734,1.023411,1.780742,2.776223,3.439482,3.419278,-0.000496,18


### Create 2D dataframe

In [6]:
start = time.time()

cols_1D = [col for col in df.columns if not any(char.isdigit() for char in col)]

df_1D = pd.concat([df[cols_1D]]*4).sort_index()

df_2D = []

for i in range(1, 5):
    cols = [col for col in df.columns if col[:6] == "point" + str(i)]
    cols_2D = [col[7:] for col in cols]
    
    df_point_i = df[cols]
    df_point_i.columns = cols_2D
    
    df_2D.append(df_point_i)

df_2D = pd.concat(df_2D, axis=0)
df_2D.sort_index(inplace=True)

df_2D = pd.concat([df_1D, df_2D], axis=1).reset_index(drop=True)

end = time.time()
print("2D df created in: {} min and {} sec".format(trunc((end - start)/60),
                                                   round((end - start)%60)))

print_ram_usage()

df_2D.tail()

2D df created in: 9 min and 56 sec
RAM usage: 85.52GB


,DATETIME,IMPACT,LATITUDE,LONGITUDE,OMF,SENSITIVITY,hour,delp_0.015,delp_0.026,delp_0.04,...,v_85.439,v_859.316,v_873.975,v_888.632,v_903.289,v_917.946,v_932.602,v_947.258,v_961.913,v_976.62
16799351,2015-02-28 18:00:00,-0.000063,89.269997,46.529999,0.084857,-0.000743,18,1.0,1.27,1.488501,...,1.590034,5.747978,5.708429,5.440742,5.119752,4.757587,4.508724,4.032255,3.689447,3.532344
16799352,2015-02-28 18:00:00,-0.000064,89.320000,-62.049999,0.129242,-0.000496,18,1.0,1.27,1.488501,...,0.721723,3.279016,2.763828,2.069474,1.507560,1.354940,1.979762,2.598012,2.769385,2.732680
16799353,2015-02-28 18:00:00,-0.000064,89.320000,-62.049999,0.129242,-0.000496,18,1.0,1.27,1.488501,...,0.762431,3.363991,2.841720,2.137918,1.564459,1.395590,2.000310,2.625103,2.801180,2.766160
16799354,2015-02-28 18:00:00,-0.000064,89.320000,-62.049999,0.129242,-0.000496,18,1.0,1.27,1.488501,...,0.773603,2.076361,1.498283,1.039572,0.803652,0.994257,1.744805,2.761100,3.437295,3.421918
16799355,2015-02-28 18:00:00,-0.000064,89.320000,-62.049999,0.129242,-0.000496,18,1.0,1.27,1.488501,...,0.720051,2.005436,1.447804,1.013766,0.806734,1.023411,1.780742,2.776223,3.439482,3.419278


### Save data

In [7]:
start = time.time()

df_2D.to_hdf("../efs/data/df_2D_4_pts.h5", key="df", complevel=9)

end = time.time()
print("2D df saved and compressed in: {} min and {} sec".format(trunc((end - start)/60),
                                                                round((end - start)%60)))

2D df saved and compressed in: 86 min and 6 sec
